In [1]:
import os
import tensorflow as tf
import numpy as np
from mlp.data_providers import MSD10GenreDataProvider, MSD25GenreDataProvider
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_data = MSD10GenreDataProvider('train', batch_size=50)
valid_data = MSD10GenreDataProvider('valid', batch_size=50)

In [3]:
def baseline_layer(inputs,state_size):
    rnn_outputs, final_state = tf.nn.dynamic_rnn(tf.nn.rnn_cell.BasicRNNCell(state_size), inputs, dtype=tf.float32)
    return rnn_outputs

In [4]:
num_segments = 120
num_dims = train_data.inputs.shape[1]/num_segments

inputs = tf.placeholder(tf.float32, [None, num_segments, num_dims], 'inputs')
targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')

state_size = train_data.num_classes
batch_size=50
with tf.variable_scope('baseline-rnn'):
    rnn_outputs = baseline_layer(inputs,state_size)
print tf.shape(rnn_outputs)
with tf.name_scope('error'):
    error = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(rnn_outputs[:,-1,:], targets))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(rnn_outputs[:,-1,:], 1), tf.argmax(targets, 1)), 
            tf.float32))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(error)
    
init = tf.global_variables_initializer()

Tensor("Shape:0", shape=(3,), dtype=int32)


In [5]:
with tf.Session() as sess:
    sess.run(init)
    for e in range(40):
        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:
            input_batch = np.reshape(input_batch, (batch_size, num_segments, num_dims))
            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy], 
                feed_dict={inputs: input_batch, targets: target_batch})
            running_error += batch_error
            running_accuracy += batch_acc
        running_error /= train_data.num_batches
        running_accuracy /= train_data.num_batches
        print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
              .format(e + 1, running_error, running_accuracy))
        if (e + 1) % 5 == 0:
            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                input_batch = np.reshape(input_batch, (batch_size, num_segments, num_dims))
                batch_error, batch_acc = sess.run(
                    [error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= valid_data.num_batches
            valid_accuracy /= valid_data.num_batches
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                   .format(valid_error, valid_accuracy))

End of epoch 01: err(train)=2.14 acc(train)=0.22
End of epoch 02: err(train)=1.97 acc(train)=0.30
End of epoch 03: err(train)=1.96 acc(train)=0.30
End of epoch 04: err(train)=1.95 acc(train)=0.30
End of epoch 05: err(train)=1.93 acc(train)=0.31
                 err(valid)=1.94 acc(valid)=0.30
End of epoch 06: err(train)=1.92 acc(train)=0.31
End of epoch 07: err(train)=1.92 acc(train)=0.31
End of epoch 08: err(train)=1.93 acc(train)=0.31
End of epoch 09: err(train)=1.92 acc(train)=0.32
End of epoch 10: err(train)=1.93 acc(train)=0.31
                 err(valid)=1.96 acc(valid)=0.30
End of epoch 11: err(train)=1.94 acc(train)=0.31
End of epoch 12: err(train)=1.93 acc(train)=0.31
End of epoch 13: err(train)=1.92 acc(train)=0.31
End of epoch 14: err(train)=1.92 acc(train)=0.32


KeyboardInterrupt: 